In [7]:
import numpy
import pickle
import csv
import os
import math
import random
import pandas as pd
import re
from gensim.models import Word2Vec

import multiprocessing as mp

# Loading in Healthcare articles

csv_file_path = "intermediate/crime.csv"

df = pd.read_csv(csv_file_path, engine='python')
df = df.dropna(subset=['Year'])
df = df.loc[(df['Year']>=2004) & (df['Year']<=2022)]
df = df.drop('Unnamed: 0', axis = 1)

0    vatican former ambassador united states quashe...
1    white house denied president trump briefed cla...
2    photo provided miami dade county corrections s...
3    republicans challenging president characteriza...
4    imposing lengthy prison terms four former empl...
Name: Text, dtype: object

In [2]:
eras = range(2004,2023)

data = []

for era in eras:
    condition = df['Year'] == era
    filtered_df = df.loc[condition, 'Text']
    x = '\n'.join(filtered_df.astype(str))
    data.append(x.split('\n'))

list_of_lists = []

In [3]:
try:
    for i in data:
        sentence_lists = []
        for article in range(len(i)):  # No need to specify 0 as start
            x = i[article].split()
            sentence_lists.append(x)
        list_of_lists.append(sentence_lists)
except Exception as e:
    print("An error occurred:", e)
    # Handle the error appropriately, e.g., logging, reducing data size, etc.
 

In [4]:
with open('intermediate/crime_corpus.pickle', 'wb') as f:
    pickle.dump(list_of_lists, f)

In [5]:
full_corpus = []
flatten = [item for sublist in list_of_lists for item in sublist]
full_corpus.extend(flatten)
del list_of_lists

In [6]:
num_cores = mp.cpu_count()

start_model = Word2Vec(full_corpus, vector_size = 300, min_count = 100, epochs = 10, sg = 1, hs = 0, negative = 5, window = 6, workers = num_cores-1)

model_name = "intermediate/crime.model"

start_model.save(model_name)